In [ ]:
%iam_role arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role

In [ ]:
%region us-west-2

In [ ]:
%additional_python_modules "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl"
%number_of_workers 2
%connections dev-glue-redshift-connection
%idle_timeout 30

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
#download configuration file
import boto3
session = boto3.Session()

s3 = session.resource('s3')
s3.Bucket('dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o').download_file('confs/leaddocket/ld_sstm.yaml', 'ld_sstm.yaml')

In [ ]:
#%stop_session

In [ ]:
with open("ld_sstm.yaml", "r") as f:
    sstm_config = f.read()
    

In [ ]:
def readFromFile(ext, path) :
    fileContent = spark.read.format(ext).option("header", "true").option("mergeSchema", "true").load(path)
    #fileContent = spark.read.format(ext).option("header", "true").load(path)
    
    return fileContent


In [ ]:
def writeToFile(df_join , path, mode) :
    df_join.write.parquet(path,mode=mode)

In [ ]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")


In [ ]:
from sstm_transformation.ld_builder import LDBuilder

In [ ]:
#Read All dfs 

df_contact =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/leaddocket/aliawadlaw/Contact/")
df_leaddetail =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/leaddocket/aliawadlaw/LeadDetail/")
df_leadrow =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/leaddocket/aliawadlaw/LeadRow/")
df_casetype =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/leaddocket/aliawadlaw/CaseType/")
df_leadsource =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/leaddocket/aliawadlaw/LeadSource//")
df_opportunities =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/leaddocket/aliawadlaw/Opportunities/")

df_referrals =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/leaddocket/aliawadlaw/Referrals/")
df_statuses =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/leaddocket/aliawadlaw/statuses/")
df_users =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/leaddocket/aliawadlaw/Users/")



In [ ]:

builder = LDBuilder("ld_sstm.yaml", spark=spark)


In [ ]:
#Upload data on S3 
from awsglue.dynamicframe import DynamicFrame
def upload_result(result):
    temp_bucket = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp"
    for key, df in result.items():
        print(f"Writing {key}")
        writeToFile(df,"s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/{}/".format(key.lower()), "overwrite")
        dyf = DynamicFrame.fromDF(dataframe=df, glue_ctx=glueContext, name=key)
        table_name = key.lower()
        dyf.show(10)
        
        datasink = glueContext.write_dynamic_frame.from_jdbc_conf(
                        frame=dyf,
                        catalog_connection="dev-glue-redshift-connection",
                        connection_options={
                            "preactions" : "delete from tsm.{}".format(table_name),
                            "dbtable": "tsm.{}".format(table_name), 
                            "database": "truve",
                        },
                        redshift_tmp_dir=temp_bucket,
                        transformation_ctx="datasink",
                    )
        '''
        #break
        redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "XXXXXXX",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf,
                                                        connection_type="redshift",
                                                        connection_options=redshift_connection_options)
        '''
        #break
    

In [ ]:
#Age Transformation
def contact_transformation(input_df):
    #Read Raw Data
    result = builder.build_contact(df=input_df)
    #print(result)
    upload_result(result)
    
contact_transformation(df_contact)

In [ ]:
#Age Transformation
def leaddetail_transformation(input_df):
    #Read Raw Data
    result = builder.build_leaddetail(df=input_df)
    #print(result)
    upload_result(result)
    
leaddetail_transformation(df_leaddetail)

In [ ]:
%stop_session

In [ ]:
df_leadrow.show(10)

In [ ]:
df_leadrow.where("id = 7385").show()

In [ ]:
#Age Transformation
def leadrow_transformation(input_df):
    #Read Raw Data
    result = builder.build_leadraw(df=input_df)
    #print(result)
    upload_result(result)
    
leadrow_transformation(df_leadrow)

In [ ]:
#Age Transformation
def casetype_transformation(input_df):
    #Read Raw Data
    result = builder.build_casetype(df=input_df)
    #print(result)
    upload_result(result)
    
casetype_transformation(df_casetype)

In [ ]:
#Age Transformation
def opportunities_transformation(input_df):
    #Read Raw Data
    result = builder.build_opportunities(df=input_df)
    #print(result)
    upload_result(result)
    
opportunities_transformation(df_opportunities)

In [ ]:
#Age Transformation
def ldsource_transformation(input_df):
    #Read Raw Data
    result = builder.build_leadsource(df=input_df)
    #print(result)
    upload_result(result)
    
ldsource_transformation(df_leadsource)

In [ ]:
#Age Transformation
def ldreferrals_transformation(input_df):
    #Read Raw Data
    result = builder.build_referrals(df=input_df)
    #print(result)
    upload_result(result)
    
ldreferrals_transformation(df_referrals)

In [ ]:
#Age Transformation
def ldstatus_transformation(input_df):
    #Read Raw Data
    result = builder.build_statuses(df=input_df)
    #print(result)
    upload_result(result)
    
ldstatus_transformation(df_statuses)

In [ ]:
df_statuses.filter("id = 5").show()

In [ ]:
#Age Transformation
def ldusers_transformation(input_df):
    #Read Raw Data
    result = builder.build_users(df=input_df)
    #print(result)
    upload_result(result)
    
ldusers_transformation(df_users)